In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import graphviz
import xgboost as xgb
import numpy as np
import os

In [9]:
# Carregar o dataset MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target
y = y.astype(int)

# Dividir o dataset em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Criar e treinar o modelo XGBoost
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=10, 
    eval_metric='mlogloss', 
    eta=0.3, 
    max_depth=6, 
    n_estimators=11,
    random_state=42)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo XGBoost: {accuracy*100:.2f}%')

Acurácia do modelo XGBoost: 94.62%


In [27]:
booster = xgb_model.get_booster()
# Definir o nome do arquivo SVG
svg_filename = 'tree.svg'

# Visualizar e salvar a árvore como SVG
plt.figure(figsize=(20, 10))
xgb.plot_tree(booster, num_trees=0)  # Ajuste num_trees para o índice da árvore desejada
plt.savefig(svg_filename, format='svg')
plt.close()

<Figure size 2000x1000 with 0 Axes>

In [34]:
format = 'svg' #You should try the 'svg'

image = xgb.to_graphviz(xgb_model)

#Set a different dpi (work only if format == 'png')
# image.graph_attr = {'dpi':'400'}

image.render('filename', format = format)


'filename.svg'

In [32]:
# Criar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
rf_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = rf_model.predict(X_test)

# Avaliar a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo Random Forest: {accuracy * 100:.2f}%')

Acurácia do modelo Random Forest: 96.74%


In [33]:
# Escolha a árvore que você deseja visualizar (por exemplo, a primeira árvore)
tree_index = 0  # O índice da árvore que você deseja visualizar
estimator = rf_model.estimators_[tree_index]

# Exportar a árvore para o formato DOT
dot_data = export_graphviz(
    estimator,
    out_file=None,
    feature_names=mnist.feature_names,
    class_names=[str(i) for i in range(10)],
    filled=True,
    rounded=True,
    special_characters=True
)

# Gerar e visualizar o gráfico usando graphviz
graph = graphviz.Source(dot_data)
graph.render('tree.svg', format='svg')  # Salva como tree.svg

'tree.svg'